In [1]:
import sys
sys.path.append('/home/mage/main/dataindex-crw')
import importlib

args = {
    "job_name": "darkness",
    "job_type": "extract",
    "option": "status_job",
    "page_type": "supplement",
    "country": "brazil",
    "mode": "dev"
}

In [4]:
job_name = args["job_name"]
page_type = args["page_type"]
country = args["country"]

module_name = f"pages.{page_type}.{country}.{job_name}.job"

# Remove o módulo do cache antes de importar novamente
if module_name in sys.modules:
    del sys.modules[module_name]

module = importlib.import_module(module_name)
module.run(args)

2024-03-03 17:58:14,100 - INFO - status_job
2024-03-03 17:58:14,101 - INFO - O arquivo None/pages/supplement/brazil/darkness/seed.json não foi encontrado.
2024-03-03 17:58:14,102 - INFO - The file None/data/supplement/brazil/darkness/tree_temp.csv does not exist
2024-03-03 17:58:14,103 - INFO - create_or_read_df
2024-03-03 17:58:14,104 - INFO - read file: None/data/supplement/brazil/darkness/origin.csv


JOB_NAME: darkness - EXEC: extract


TypeError: 'NoneType' object is not iterable